### 네트워크 플로우 기본 문제
- 단, 동일한 간선이 여러개 들어오면 용량을 추가로 늘려줘야 하는 조건이 추가됐다.

### Ford Fulkerson 알고리즘

In [ ]:
import sys
input= sys.stdin.readline

def dfs(capacity, flow, vis, vertices, u, sink, cur= 1 << 63) :
  if u == sink: return cur

  vis[u] = True
  for v in range(vertices) :
    if vis[v] or flow[u][v] >= capacity[u][v] : continue # 방문했거나, 용량보다 흘릴 양이 더 많으면 패스
    available_flow = min(cur, capacity[u][v] - flow[u][v]) # 여유 용량
    tmp = dfs(capacity, flow, vis, vertices, v, sink, available_flow)
    if tmp :
      flow[u][v] += tmp
      flow[v][u] -= tmp # 역방향 간선은 같은 크기의 음의 유량을 흘려준다.
      return tmp
  return 0

def fulkerson(capacity, source, sink) :
  V = len(capacity)
  flow = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    vis = [False] * V
    cur = dfs(capacity, flow, vis, V, source, sink) # dfs를 통해 흘릴 수 있는 양을 구한다.
    if cur == 0 : break
    ret += cur

  return ret

def convert(x) :
  if x <= ord('Z') : return x - ord('A')
  return x - ord('a') + 26

def sol() :
  N = int(input())
  V = 52
  C = [[0] * V for _ in range(V)] # capacity

  for _ in range(N) :
    u, v, c = input().split()
    u, v, c = convert(ord(u)), convert(ord(v)), int(c)
    C[u][v] += c # 병렬로 이어진 경우 누적해야 하므로 +=
    C[v][u] += c

  ans = fulkerson(C, 0, 25)
  print(ans)
sol()

### Edmonds-Karp 알고리즘
- 뭔가 재귀보다 좀 더 느리다

In [ ]:
import sys, collections
input= sys.stdin.readline

def edmonds(capacity, source, sink) :
  V = len(capacity)
  flow = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    tmp = 0
    Q = collections.deque()
    vis = [False] * V
    P = [-1] * V
    vis[source] = True
    Q.append((source, 1 << 63))

    while Q :
      front = Q.popleft()
      u, current_flow = front
      if u == sink :
        tmp = current_flow
        break
      for v in range(V) :
        if vis[v] or flow[u][v] >= capacity[u][v] : continue
        vis[v] = True
        P[v] = u
        Q.append((v, min(current_flow, capacity[u][v] - flow[u][v])))

    if P[sink] == -1: break
    ret += tmp
    p = P[sink]
    u = sink
    while p != -1 :
      flow[p][u] += tmp
      flow[u][p] -= tmp
      u = p
      p = P[u]
  return ret

def convert(x) :
  if x <= ord('Z') : return x - ord('A')
  return x - ord('a') + 26

def sol() :
  N = int(input())
  V = 52
  C = [[0] * V for _ in range(V)] # capacity

  for _ in range(N) :
    u, v, c = input().split()
    u, v, c = convert(ord(u)), convert(ord(v)), int(c)
    C[u][v] += c # 병렬로 이어진 경우 누적해야 하므로 +=
    C[v][u] += c

  ans = edmonds(C, 0, 25)
  print(ans)
sol()

### Dinic 알고리즘
- 코드 최적화를 좀 더 진행해야 할 것 같다.

In [ ]:
import sys, collections
input= sys.stdin.readline

def bfs(capacity, flow, level, source, sink) :
  """꽉차지 않은 간선만으로 sink에 도달할 수 있는지 확인한다."""
  vertices = len(capacity)
  Q = collections.deque()
  Q.append(source)
  level[source] = 0
  while Q :
    u = Q.popleft()
    for v in range(vertices) :
      if level[v] == -1 and capacity[u][v] - flow[u][v] > 0 :
        level[v] = level[u] + 1
        Q.append(v)
  return level[sink] != -1

def dfs(capacity, flow, work, level, u, sink, current_flow = 1 << 63) :
  """꽉차지 않은 flow 간선을 찾는다"""
  if u == sink : return current_flow
  vertices = len(capacity)
  while work[u] < vertices :
    v = work[u]
    if level[v] == level[u] + 1 and capacity[u][v] > flow[u][v] :
      available_flow = min(current_flow, capacity[u][v] - flow[u][v])
      tmp = dfs(capacity, flow, work, level, v, sink, available_flow)
      if tmp > 0 :
        flow[u][v] += tmp
        flow[v][u] -= tmp
        return tmp
    work[u] += 1
  return 0

def dinic(capacity, source, sink) :
  vertices = len(capacity)
  flow = [[0] * vertices for _ in range(vertices)]
  ret = 0

  while True :
    level = [-1] * vertices
    work = [0] * vertices
    if not bfs(capacity, flow, level, source, sink) : break
    while True:
      tmp = dfs(capacity, flow, work, level, source, sink)
      if tmp == 0 : break
      ret += tmp
  return ret

def convert(x) :
  if x <= ord('Z') : return x - ord('A')
  return x - ord('a') + 26

def sol() :
  N = int(input())
  V = 52
  C = [[0] * V for _ in range(V)] # capacity

  for _ in range(N) :
    u, v, c = input().split()
    u, v, c = convert(ord(u)), convert(ord(v)), int(c)
    C[u][v] += c # 병렬로 이어진 경우 누적해야 하므로 +=
    C[v][u] += c

  ans = dinic(C, 0, 25)
  print(ans)
sol()